<div align="center"> 
<img src='./img/header.png'/>
</div>

## [Global Ice Velocities](https://its-live.jpl.nasa.gov/)
    
The Inter-mission Time Series of Land Ice Velocity and Elevation (ITS_LIVE) project facilitates ice sheet, ice shelf and glacier research by providing a globally comprehensive and temporally dense multi-sensor record of land ice velocity and elevation with low latency.

Scene-pair velocities generated from satellite optical and radar imagery.

* Coverage: All land ice
* Date range: 1985-present
* Resolution: 240m
* Scene-pair separation: 6 to 546 days


---

* If you want to query our API directly using  your own software here is the OpenApi endpoint https://staging.nsidc.org/apps/itslive-search/docs
* For questions about this notebook and the dataset please contact users services at uso@nsidc.org


In [3]:
#1: Now let's render our UI and pick up an hemisphere, if you update the hemisphere you need to execute the cell again.
import warnings
warnings.filterwarnings('ignore')
from itslive import itslive_ui
ui = itslive_ui('north')
ui.render()

Dropdown(description='Hemisphere:', index=2, options=('global', 'south', 'north'), value='north')

SelectionRangeSlider(description='Date Range', index=(0, 13149), layout=Layout(width='100%'), options=((' 1984…

Map(center=[90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [4]:
#2: We build the parameters to query the ITS_LIVE Search API, we get the time coverage for our selected area
params = ui.build_params()
print(f'current parameters: {params}')
timeline = None
if params is not None:
    timeline = ui.update_coverages()
    total =  sum(item['count'] for item in timeline)
    print(f'Total data granules: {total:,}')
timeline

current parameters: {'polygon': '-15.1749,81.3293,-15.1749,81.3313,-15.1729,81.3313,-15.1729,81.3293,-15.1749,81.3293', 'start': '1984-01-01', 'end': '2020-01-01', 'percent_valid_pixels': 1}
Total data granules: 56,538


[{'count': 2, 'year': 1984},
 {'count': 129, 'year': 1985},
 {'count': 244, 'year': 1986},
 {'count': 373, 'year': 1987},
 {'count': 1890, 'year': 1988},
 {'count': 2091, 'year': 1989},
 {'count': 1455, 'year': 1990},
 {'count': 4409, 'year': 1991},
 {'count': 4401, 'year': 1992},
 {'count': 4075, 'year': 1993},
 {'count': 333, 'year': 1994},
 {'count': 1510, 'year': 1995},
 {'count': 2, 'year': 1996},
 {'count': 1285, 'year': 1997},
 {'count': 748, 'year': 1998},
 {'count': 6, 'year': 2000},
 {'count': 14, 'year': 2001},
 {'count': 5, 'year': 2002},
 {'count': 28, 'year': 2003},
 {'count': 704, 'year': 2004},
 {'count': 312, 'year': 2005},
 {'count': 1183, 'year': 2006},
 {'count': 1155, 'year': 2007},
 {'count': 550, 'year': 2008},
 {'count': 609, 'year': 2009},
 {'count': 668, 'year': 2010},
 {'count': 54, 'year': 2011},
 {'count': 51, 'year': 2012},
 {'count': 2419, 'year': 2013},
 {'count': 1296, 'year': 2014},
 {'count': 3174, 'year': 2015},
 {'count': 5226, 'year': 2016},
 {'cou

In [5]:
#3: Now we are going to get the velocity pair urls, this does not download the files yet just their location
urls = []
params = ui.build_params()
if params is not None:
    urls = ui.get_granule_urls(params)
    # Print the first 10 granule URLs
    for url in urls[0:10]:
        print(url)

{'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32628/LT05_L1TP_016001_19850821_20170218_01_T1_X_LT05_L1GS_016246_19850602_20170219_01_T2_G0240V01_P030.nc'}
{'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32627/LT05_L1TP_003248_19890501_20170203_01_T1_X_LT05_L1TP_003248_19880412_20170209_01_T1_G0240V01_P033.nc'}
{'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32627/LT05_L1TP_003248_19900520_20170130_01_T2_X_LT05_L1TP_003248_19890501_20170203_01_T1_G0240V01_P040.nc'}
{'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32627/LT05_L1TP_003248_19900605_20170129_01_T1_X_LT05_L1TP_003248_19890501_20170203_01_T1_G0240V01_P038.nc'}
{'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32627/LT05_L1TP_003248_19920525_20180220_01_T2_X_LT05_L1TP_017001_19920425_20180220_01_T2_G024

## Filtering granules by year and month

In [ ]:
#4: This will query AWS(where the granules are stored) so we know the total size of our first N granules
# This may take some time, try reducing the selected area or constraining the other parameters to download a reasonable number of granules.
url_list = [url['url'] for url in urls]
# urls
filtered_urls = ui.filter_urls(url_list, max_files_per_year=5, months=[3,4,5,6,7])

# max_granules = 100
# sizes = ui.calculate_file_sizes(filtered_urls, max_granules)
# total_zise = round(sum(sizes)/1024,2)
# print(f'Approx size to download for the first {max_granules:,} granules: {total_zise} MB')
print(len(filtered_urls))
filtered_urls

## Downloading the data
**Now that we have our list of data granules we can download them from AWS.**

If this notebook is running inside AWS we could load the granules into a Dask cluster and reduce our processing times and costs.
Let's get some coffee, some data requests are in the Gigabytes realm and may take a little while to be processed. 
Once that your status URL says is completed we can grab the HDF5 data file using the URL on the same response!

In [ ]:
#5 This will download the first 50 velocity pairs
files = ui.download_velocity_pairs(filtered_urls, start=0, end=50)
files

## Working with the data

In [ ]:
import os
import glob
import xarray as xr
import pyproj
import warnings
import pandas as pd
import numpy as np
from datetime import datetime
from shapely.geometry import Polygon
warnings.filterwarnings('ignore')
centroid = ui.dc.last_draw['geometry']['coordinates']
# coord = [-49.59321, 69.210579]
#loads an array of xarray datasets from the nc files
velocity_pairs = ui.load_velocity_pairs('data')

centroid

In [ ]:
velocities = []
mean_offset_meters = 1200
for ds in velocity_pairs:
    proj = str(int(ds.UTM_Projection.spatial_epsg))
    selected_coord = ui.transform_coord('4326',proj, centroid[0], centroid[1])
    projected_lon = round(selected_coord[0])
    projected_lat = round(selected_coord[1])
    mid_date = datetime.strptime(ds.img_pair_info.date_center,'%Y%m%d')
    # We are going to calculate the mean value of the neighboring pixels(each is 240m) 10 x 10 window
    mask_lon = (ds.x >= projected_lon - mean_offset_meters) & (ds.x <= projected_lon + mean_offset_meters)
    mask_lat = (ds.y >= projected_lat - mean_offset_meters) & (ds.y <= projected_lat + mean_offset_meters)
    v = ds.where(mask_lon & mask_lat , drop=True).v.mean(skipna=True)
    # If we have a valid value we add it to the velocities array.
    if not np.isnan(v):
        velocities.append({'date': mid_date, 'mean_velocity': v.values.ravel()[0]})
velocities

In [ ]:
import pandas as pd
# order by date
df = pd.DataFrame(velocities)
df = df.sort_values(by='date').reset_index(drop=True)
df

In [ ]:
df.plot(x ='date', y='mean_velocity', kind = 'bar')

In [ ]:
velocity_pairs[0].plot.scatter(x='x', y='y', hue='v')